In [1]:
!pip install transformers datasets torch evaluate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertModel, AutoModelForCausalLM, pipeline
from datasets import Dataset
from google.colab import drive, userdata
import torch
import numpy
from huggingface_hub import login
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
orientation_data = pd.read_csv("/content/orientation-lv-train.tsv", sep="\t")
dataset = Dataset.from_pandas(orientation_data)


In [4]:
login(userdata.get('h_token'))

In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", offload_folder="Auto")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", offload_folder="auto", device_map="auto", torch_dtype=torch.float16)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
def inference(batch, lang="text"):
    results = []
    for text in tqdm(batch[lang], desc=f"Classifying texts ({lang})"):
        prompt = (
            f"Classify the following political statement into one of two categories: 'left-leaning' or 'right-leaning'. "
            f"Respond with only the category, and say 'the answer is 'right-leaning.' or 'the answer is 'left-leaning.'."
            f"Statement: {text}\n\n"
            f"Answer:"
        )

        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.7, num_return_sequences=1)

        raw_prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        prediction = raw_prediction.split("Answer:")[1].strip().lower()

        if "left-leaning" in prediction.lower():
            results.append(0)
        else:
            results.append(1)

    return {"predictions": results}

en_predictions = []
batch_size = 10

for i in range(0, len(dataset), batch_size):
    batch = dataset[i : i + batch_size]
    predictions = inference(batch, "text_en")
    en_predictions.extend(predictions["predictions"])

lv_predictions = []
for i in range(0, len(dataset), batch_size):
    batch = dataset[i : i + batch_size]
    predictions = inference(batch)
    lv_predictions.extend(predictions["predictions"])




Classifying texts (text): 100%|██████████| 8/8 [00:16<00:00,  2.05s/it]


In [8]:
print("Classification Report for Latvian:")
print(classification_report(dataset["label"], lv_predictions))

print("Classification Report for English:")
print(classification_report(dataset["label"], en_predictions))

Classification Report for Latvian:
              precision    recall  f1-score   support

           0       0.24      0.72      0.36       170
           1       0.84      0.39      0.53       628

    accuracy                           0.46       798
   macro avg       0.54      0.55      0.45       798
weighted avg       0.71      0.46      0.50       798

Classification Report for English:
              precision    recall  f1-score   support

           0       0.23      0.74      0.35       170
           1       0.82      0.32      0.46       628

    accuracy                           0.41       798
   macro avg       0.53      0.53      0.41       798
weighted avg       0.70      0.41      0.44       798
